# Objetivo

Em um jogo, treinar um taxi para levar um passageiro de um ponto a outro da maneira mais rápida possível.

# Criando o ambiente

In [4]:
import gym #biblioteca com funções e ambientes para treinamento por reforço
import random

random.seed(1234) #padronizar os valores aleatórios entre runtimes diferentes

streets = gym.make("Taxi-v3").env #cria o ambiente com o taxi, pontos e ruas
streets.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Resumo do significado de cada parte:
1. As letras são pontos de destino e busca de passageiro.
2. A letra azul é onde o táxi deve buscar o passageiro.
3. A letra roxa é para onde o táxi deve levar o passageiro.
4. As linhas | representam paredes pelas quais o táxi não consegue passar.
5. O retângulo preenchido é o táxi, que está amarelo quando vazio, e verde quando tem passageiro.

O ambiente é 5x5 (25 localizações possíveis)
O que define um estado do ambiente:
1. Onde o táxi está (25 possibilidades).
2. Onde o atual destino é (4 possibilidades).
3. Onde o passageiro está (5 possibilidades, nas letras e dentro do táxi).

Ou seja, há 25 x 4 x 5 = 500 estados possíveis nesse ambiente.

Para cada estado, há 6 possíveis ações:
1. O táxi se mover para leste, oeste, norte ou sul.
2. Pegar um passageiro.
3. Largar um passageiro.

Os valores Q serão dados pelos seguintes critérios:
1. Largar o passageiro no local correto dá +20 pontos ao atual par (estado, ação).
2. Todo (estado, ação) enquanto carregando um passagerio resulta em -1 ponto pro (estado, ação). Isso incentiva o encurtamento dos caminhos.
3. Pegar ou largar um passageiro no lugar errado dá uma penalidade de -10 pontos para o (estado,ação)

Vamos começar colocando o táxi na localização (2, 3), com o passageiro na localização de busca 2, e o destino na localização 0:

In [5]:
initial_state = streets.encode(2, 3, 2, 0) #forma um objeto de estado com esses parâmetros
streets.s = initial_state #assinala o estado atual do ambiente streets para esse
streets.render() #imprime o ambiente em seu estado atual

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [7]:
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

Essa é o dicionário de recompensa para cada 1 das 6 ações possíveis nesse estado(ir a sul, leste, oeste, norte, largar o passageiro, pegar o passageiro), e cada uma tem associada a ela uma tupla com valores que indicam: a probabilidade de se tomar aquela ação, o estado em que o ambiente ficaria após ela (dos 500 possíveis), o valor Q adicionado ao par (estado, ação) e se essa ação resultaria no cumprimento da missão (largar o passageiro na letra roxa).

Agora vamos programar a Q-Learning, com 10000 simulações de corridas do táxi, avançando um passo no tempo em cada corrida, com 10% de chance do taxi tomar uma decisão aleatória/exploratória ao invés de escolher a com maior valor Q:

In [15]:
import numpy as np

q_table = np.zeros([streets.observation_space.n, streets.action_space.n]) #array numpy que contém todas as combinações de estados e ações no ambiente, com seus valores inicializados para 0

learning_rate = 0.1 
discount_factor = 0.6 #fator de "trigger-back" do valor de uma ação para as anteriores
exploration = 0.1 #chance de se tomar uma ação aleatória não baseada nos valores Q
epochs = 10000 #número de corridas

for taxi_run in range(epochs):
    state = streets.reset() #reseta o ambiente para uma nova corrida
    done = False
    
    while not done:
        random_value = random.uniform(0, 1) #gera um float aleatória entre 0 e 1
        if (random_value < exploration):
            action = streets.action_space.sample() #explora uma ação aleatória
        else:
            action = np.argmax(q_table[state]) #toma a ação com o maior valor Q
            
        next_state, reward, done, info = streets.step(action) #aplica a ação e retorna os resultados
        
        prev_q = q_table[state, action]
        next_max_q = np.max(q_table[next_state])
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q) #equação da Q-Learning, de "trigger-back"
        q_table[state, action] = new_q #atualiza a tabela de valores Q para esse par de estado e ação
        
        state = next_state #passa para o próximo estado, resultando da última ação
        
        

In [16]:
q_table[initial_state]

array([-2.42376178, -2.41530761, -2.39404973, -2.3639511 , -9.0034776 ,
       -8.06209957])

Esses são os valores associados a cada ação possível no estado inicial baseados nos valores aprendidos no bloco anterior, pela Q-Learning.

## Testando os valores Q encontrados:

In [18]:
from IPython.display import clear_output
from time import sleep

for tripnum in range(1, 11): #teste com 10 corridas
    state = streets.reset() #reseta o ambiente para uma nova corrida
   
    done = False
    trip_length = 0
    
    while not done and trip_length < 25: #limita a corrida a 25 passos ou a quando o passageiro é largado no ponto correto
        action = np.argmax(q_table[state]) #toma a decisão de maior valor Q
        next_state, reward, done, info = streets.step(action) #toma a ação e captura os dados das consequências
        clear_output(wait=True)
        print("Trip number " + str(tripnum) + " Step " + str(trip_length))
        print(streets.render(mode='ansi'))
        sleep(.5)
        state = next_state #passa para o próximo estado, consequência da última ação
        trip_length += 1
        
    sleep(2)
    

Trip number 10 Step 24
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

